# NEXUS tool: case study for the NWSAS basin - energy demand calculations
In this notebook a case study for the NWSAS basin is covered using the `nexus_tool` package. The water requirements for agricultural irrigation were previously calculated (check the water demand case study). In this case study, the energy requirements for pumping and desalination of brackish water are estimated.

First import the package by running the following block:

In [1]:
%load_ext autoreload

In [11]:
%autoreload
import sys
sys.path.append("..") #this is to add the avobe folder to the package directory
import nexus_tool

After importing all required packages, the input GIS data is loaded into the variable `df`. Change the `file_path` variable to reflect the name and relative location of your data file. This dataset should already have the water demand for irrigation data.

In [3]:
file_path = r'nwsas_results/water_demand.csv'
df = nexus_tool.read_csv(file_path)
df['Pipe_diameter']=10 #in m
df['Pipeline_length']=200 #in m

## 1. Calculating pumping energy requirements
### 1.1. Creating the model
To create a model simply create an instance of the `nexus_tool.Model()` class and store it in a variable name. The `nexus_tool.Model()` class requires a dataframe as input data. Several other properties and parameter values can be defined by explicitly passing values to them. To see a full list of parameters and their explaination refer to the documentation of the package.

In [4]:
nwsas = nexus_tool.Model(df)

### 1.2. Changing default values for inputs
The names of the properties of the model can be chaned at any time. This is important for the model to know how each property is called withing your input data. To check the current property names run the `nwsas.print_properties()` method, a list with the names of each property and its current value wil be displayed:

In [5]:
nwsas.print_properties()

Properties names:
    - Reference evapotranspiration (.eto): ETo_
    - Latitude (.lat): lat
    - Elevation (.elevation): elevation
    - Wind speed (.wind): wind
    - Solar radiation (.srad): srad
    - Min temperature (.tmin): tmin
    - Max temperature (.tmax): tmax
    - Avegarage temperature (.tavg): tavg
    - Cropland share column (.crop_share): crop_share
    - Cropland area column (.crop_area): crop_area
    - Harvest seasons names (.seasons): ['init', 'dev', 'mid', 'late']
    - Seasson start suffix (.start): _start
    - Seasson end suffix (.end): _end
    - Cropland column (.crop_column): crop
    - Groundwater table depth (.gw_depth): gw_depth
    - Total dynamic head ground water (.tdh_gw): tdh_gw
    - Total dynamic head surface water (.tdh_sw): tdh_sw


Then you can provide the right names for each property, calling them and assigning a value as:
```python
nwsas.crop_column = 'cropland'
nwsas.seasons = ['initial', 'development', 'mid-season', 'harvest']
nwsas.gw_depth = 'name_of_ground_water_depth'
```

### 1.3. Calculating the Total Dinamic Head (tdh)
First, we will calculate the Total Dinamic Head (tdh), which is a measure in meters that accounts for the depth to groundwater, the water drawdown (wdd), the operating application pressure (oap) and the pressure loss in distribution (pld).

For that, run the `nwsas.get_gw_tdh()` method passing the `wdd`, `oap` and `pld` values (their default value is 0). Also, make sure that the name for the groundwater depth column of the dataframe, matches the name of the `gw_depth` variable in the model as described in the previous step.

In [6]:
nwsas.get_gw_tdh(inplace = True, wdd = 0, oap = 0, pld = 0)

In [9]:
nwsas.get_A(inplace=True)
nwsas.get_V(inplace=True)
nwsas.get_Re(inplace=True)
nwsas.get_f(inplace=True)

nwsas.get_sw_tdh(inplace = True)

### 1.4. Calculating the energy efficiency of the pumping system
To second step compute the energy requirements for pumping water from a groundwater aquifer, first the pumping system efficiency needs to be defined:

$$\text{pumping plant efficiency (%)}= \text{fuel efficiency (%)} \times \text{power unit eff (%)} \times  \text{transmission eff (%)} \times \text{pump eff (%)} \times 100\%$$

**The Power Unit:** can be diesel engine or electric engine. In the first we call it (diesel powered pump) and the second (electric powered pump).
 
Value obtained from FAO1992, pages 26-27:

* Diesel powered pump:
    * Worst case: $0.9\times 0.3\times 0.9\times 0.4 = 10\%\ (0.1)$
    * Best case: $1\times 0.4\times 1\times 0.8 = 32\%\ (0.32)$
    
* Electric powered pump:
    * Worst case: $0.9\times 0.75\times 0.9\times 0.4 = 25\%\ (0.25)$
    * Best case: $1\times 0.85\times 1\times0.8 = 70\%\ (0.7)$

The $\text{fuel efficiency}$ and the $\text{power unit efficiency}$ values can be directly passed to the `.get_pumping_energy()` method, whereas the remaining two efficiency values can be accounted later in the least-cost supply options calculations.

After all required inputs and parameters are computed, the `nwsas.get_GWpumping_energy()` method can used to compute the groundwater pumping energy requirements.

In [12]:
nwsas.trans_eff = 0.9
nwsas.pump_eff = 0.4
nwsas.get_GWpumping_energy(inplace = True)
nwsas.get_SWpumping_energy(inplace = True)
nwsas.get_total_pumping_energy(inplace = True)
nwsas.get_annual_electricity(inplace = True)

## 2. Displaying and saving the results
After the calculations are completed, display a summary of results by running the `nwsas.print_summary()` method. If you run the method without any argument, then the summary values will be dispayed for the entire region, under the label of "Glogal". However, if you like to summarize by regions, then pass the argument `geo_boundary` to the function, specifing the variable that you want the results to be grouped by. 

In [13]:
nwsas.print_summary(geo_boundary = 'province')

,Irrigated area (ha),Water demand (Mm3),Water intensity (m3/ha),Energy demand (GWh)
province,,,,
Adrar,24101.816848,380.659062,15793.791148,649.340755
Biskra,5749.995666,57.693849,10033.720376,54.541856
Djelfa,3449.972096,35.546265,10303.348469,87.876094
El Oued,68999.985344,791.446126,11470.236142,435.985168
Gabes,2874.983928,28.963020,10074.150089,24.976582
Ghadamis,5353.213079,56.291559,10515.471347,169.482599
Ghardaia,34499.998422,440.742945,12775.158413,893.124973
Gharyan,8557.616421,92.582518,10818.727280,233.882444
Illizi,1770.396215,25.085808,14169.601170,54.883742


Finally, save the results in .csv format, by specifing an output file name and location (`output_file`) and running the `nwsas.df.to_csv(output_file, index = False)` method.

In [14]:
output_file = r'nwsas_results/pumping_energy.csv'
nwsas.df.to_csv(output_file, index = False)